## Binary Classification (Logistic Regression) - Breast Cancer [numpy]

- created: 2024.11.17

### Data

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data
x, y = load_breast_cancer(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

print(x_train_scaled.shape, y_train.shape)
print(x_test_scaled.shape, y_test.shape)

(455, 30) (455,)
(114, 30) (114,)


### [scikit-learn] Modeling and Training

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# sklearn model
model = LogisticRegression()
model.fit(x_train_scaled, y_train)

# Training
y_pred = model.predict(x_train_scaled)
acc = accuracy_score(y_train, y_pred)
print(f'Train Accuracy: {acc:.4f}')

# Evaluation
y_pred = model.predict(x_test_scaled)
acc = accuracy_score(y_test, y_pred)
print(f'Test  Accuracy: {acc:.4f}')

Train Accuracy: 0.9868
Test  Accuracy: 0.9737


### [numpy] Modeling and Training

- Manual backward propagation
- Manual update of weights and biases

In [3]:
import numpy as np
from scipy.special import expit as sigmoid

def bce_loss(y_pred, y_true):
    eps = 1e-8
    return -np.mean(y_true*np.log(y_pred + eps) + (1 - y_true)*np.log(1 - y_pred + eps))

def binary_accuracy(y_pred, y_true):
    y_pred = (y_pred > 0.5).astype(int)
    return np.mean(y_pred == y_true)

## Data
x, y = x_train_scaled, y_train.reshape(-1, 1)
print(x.shape, y.shape)

(455, 30) (455, 1)


In [4]:
## Model: 2-layer MLP
np.random.seed(42)
input_size, hidden_size, output_size = 30, 100, 1

w1 = np.random.randn(input_size, hidden_size)   # weight of 1st layer
b1 = np.zeros(hidden_size)                      # bias of 1st layer
w2 = np.random.randn(hidden_size, output_size)  # weight of 2nd layer
b2 = np.zeros(output_size)                      # bias of 2nd layer

## Train
n_epochs = 10000
learning_rate = 0.01

for epoch in range(1, n_epochs + 1):
    # Forward propagation
    z1 = np.dot(x, w1) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(a1, w2) + b2
    out = sigmoid(z2)

    loss = bce_loss(out, y)
    score = binary_accuracy(out, y)

    # Backward progapation
    grad_out = (out - y) / out / (1 - out) / y.shape[0]
    grad_z2 = out * (1 - out) * grad_out
    grad_w2 = np.dot(a1.T, grad_z2)
    grad_b2 = np.sum(grad_z2, axis=0)

    grad_a1 = np.dot(grad_z2, w2.T)
    grad_z1 = a1 * (1 - a1) * grad_a1
    grad_w1 = np.dot(x.T, grad_z1)
    grad_b1 = np.sum(grad_z1, axis=0)

    # Update weights and biases
    w1 -= learning_rate * grad_w1
    b1 -= learning_rate * grad_b1
    w2 -= learning_rate * grad_w2
    b2 -= learning_rate * grad_b2

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch}/{n_epochs}] loss: {loss.item():.2f} score: {score:.4f}")

[1000/10000] loss: 0.12 score: 0.9516
[2000/10000] loss: 0.08 score: 0.9692
[3000/10000] loss: 0.07 score: 0.9802
[4000/10000] loss: 0.06 score: 0.9868
[5000/10000] loss: 0.05 score: 0.9890
[6000/10000] loss: 0.04 score: 0.9890
[7000/10000] loss: 0.04 score: 0.9890
[8000/10000] loss: 0.04 score: 0.9890
[9000/10000] loss: 0.03 score: 0.9912
[10000/10000] loss: 0.03 score: 0.9934
